In [1]:
import os
import json
import requests
from web3 import Web3
from dotenv import load_dotenv

load_dotenv()

def load_abi(filename):
    with open(filename, "r") as f:
        abi = json.load(f)
    return abi

rpc_url = "https://poly-rpc.gateway.pokt.network/"
web3 = Web3(Web3.HTTPProvider(rpc_url))
block_number = web3.eth.blockNumber
print(f"Connected: {web3.isConnected()}, block number: {block_number}")

contract_address = Web3.toChecksumAddress("0xb6A5D547d0A325Ffa0357E2698eB76E165b606BA")
abi_filename = (
    "../resources/71937/Devcon_Offset_Pool_0xb6A5D547d0A325Ffa0357E2698eB76E165b606BA.json"
)

abi = load_abi(abi_filename)
pooling_contract = web3.eth.contract(address=contract_address, abi=abi)
contributor_addresses = pooling_contract.functions.getContributorsAddresses().call()
contributed_nct = pooling_contract.functions.totalCarbonPooled().call()

print("Contributor count: ", len(contributor_addresses))
print(f"Amount of NCT contributed: {web3.fromWei(contributed_nct, 'ether'):.2f} (Tonnes of CO2)")

if not os.path.exists("images"):
    os.mkdir("images")

Connected: True, block number: 36536277
Contributor count:  228
Amount of NCT contributed: 453.21 (Tonnes of CO2)


## Voting  

In [9]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

snapshot_csv = "./snapshot-report-0xf7b49d23003b387cbbdf7e35f779067a95104a74f79531462448aaa33696d3d0.csv"

chunks = pd.read_csv(snapshot_csv, chunksize=100000)
all_snapshot_data = pd.concat(chunks)
# Filter the vote data by addresses that contributed to climate friendly devcon
eligible_snapshot_data = all_snapshot_data[all_snapshot_data.address.isin(contributor_addresses)]
print("Eligible voter count", len(eligible_snapshot_data))

vote_choices = eligible_snapshot_data["choice"].map({1: "Cordillera Azul, Peru (2013)", 2: "Rimba Raya, Indonesia (2014)", 3: "North Pikounda, DR Congo (2012)"})
s = eligible_snapshot_data.choice
fig = px.pie(
    vote_choices, 
    values=vote_choices.value_counts().values, 
    names=vote_choices.value_counts().index,
    hole=0.33,
    color_discrete_sequence=px.colors.qualitative.Prism[2:5])
fig.update_traces(hoverinfo="label+percent", textinfo="label+percent", textposition="outside")
fig.update_layout(title_text="Choice of Carbon Offset Project to make Devcon VI Climate Friendly", 
    title_x=0.5,
    title={'font': {'size': 22}},
    font= {'size': 16},
    width=1000, 
    height=700,
    showlegend=False)
fig.show()

fig.write_image("images/Vote.png")

Eligible voter count 59


## Token Participation

In [10]:
api_key = os.environ.get("POLYGONSCAN_API_KEY")
if not api_key:
    print("error: please add your polygonscan api key to the .env")
deployment_block_number = 33870323
current_block_number = web3.eth.blockNumber
base_url = "https://api.polygonscan.com/api?"
params = (f"module=account"
            f"&action=txlist"
            f"&address={contract_address}"
            f"&startblock={deployment_block_number}"
            f"&endblock={current_block_number}"
            f"&sort=asc"
            f"&apikey={api_key}")
print("Getting...", base_url + params)
response = requests.get(base_url + params)
print(response.status_code)
print(response.content)
assert response.status_code == 200, f"Oops, request unsuccessful, {response.reason}"

Getting... https://api.polygonscan.com/api?module=account&action=txlist&address=0xb6A5D547d0A325Ffa0357E2698eB76E165b606BA&startblock=33870323&endblock=36536686&sort=asc&apikey=EKF6HRGDCSHWZFCCUSTDX2FI7Z385HY5FI
200
b'{"status":"1","message":"OK","result":[{"blockNumber":"33870323","timeStamp":"1664783423","hash":"0x1fcae238063fceb587e7af2516bb21f9c0ea0c098c3c05d3b699df6d7c988960","nonce":"95","blockHash":"0x4d5e95198bf32de1c0689407071e6ce1c29c150cd6bc7730b16e7cc147dee2df","transactionIndex":"60","from":"0x967af011954f71835167e88b61226b96cd558896","to":"","value":"0","gas":"1826195","gasPrice":"31500000014","isError":"0","txreceipt_status":"1","input":"0x6080604052600060025573439d22a39cd2d0f0572ed05d3c5081d6117b8031600360006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550731b02da8cb0d097eb8d57a175b88c7d8b47997506600460006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373fffffffffffffffffffffffffffff

In [4]:
response_content = json.loads(response.content)
transaction_list = response_content['result']
print("Transaction count", len(transaction_list))

erc20_names = {
    "0xD838290e877E0188a4A44700463419ED96c16107": "NCT",
    "0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174": "USDC",
    "0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063": "DAI",
    "0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619": "WETH",
    "0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270": "WMATIC",
    }

erc20_count = 0
matic_count = 0
matic_sent = 0
txn_error_count = 0
data = []
for txn in transaction_list:
    if int(txn["isError"]):
        txn_error_count += 1
        continue
    if txn['functionName'].startswith("participateWithToken"):
        erc20_count += 1
        for erc20_address, erc20_name in erc20_names.items():
            #amount_nct = 
            if erc20_address == Web3.toChecksumAddress(txn["input"][34:74]):
                if erc20_name == "USDC":
                    decimals = 6
                else:
                    decimals = 18
                data.append(
                    {"token": erc20_name, 
                     "amount": int(txn['value'])/10**decimals,
                     "blocknumber": int(txn['blockNumber']),
                     "timestamp": int(txn["timeStamp"]),
                     "nonce": int(txn["nonce"]),
                     "from": Web3.toChecksumAddress(txn["from"]),
                     "amount NCT": None
                    }
                    )
    elif txn['functionName'].startswith("participateWithMatic"):
        data.append(
            {"token": "MATIC", 
             "amount": int(txn['value'])/10**18,
             "blocknumber": int(txn['blockNumber']),
             "timestamp": int(txn["timeStamp"]),
             "nonce": int(txn["nonce"]),
             "from": Web3.toChecksumAddress(txn["from"]),
             "amount NCT": None
            }
        )
        matic_sent += int(txn['value'])
    else:
        print(txn['functionName'])
print("txn_error", txn_error_count)
df = pd.DataFrame(data)

Transaction count 240
atInversebrah(int248 a, uint48[] b, uint32 c, bytes20[] d, bytes30[] e)
txn_error 0


In [11]:
import plotly.express as px
fig = px.pie(df.token, values=df.token.value_counts().values, names=df.token.value_counts().index)
#fig.update_traces(hoverinfo='label+percent', textinfo='token')
fig.show()
fig.write_image("images/Tokens.png")

In [8]:
from datetime import datetime
# print(data[:10])

contribution_dates = []

for entry in data:
    date = (datetime.utcfromtimestamp(int(entry["timestamp"])).strftime("%Y-%m-%d"))
    # print(date)
    contribution_dates.append(date)

dates_count = dict( (l, contribution_dates.count(l) ) for l in set(contribution_dates))
# print(dates_count)
dates_count_df = pd.DataFrame.from_dict(dates_count, orient='index')
dates_count_df.index.names = ['Date']
dates_count_df.rename(columns = {0:'Participants'}, inplace = True)

# print(dates_count_df.sort_index())

# fig = px.bar(dates_count, x='year', y='pop')
# fig.show()

fig = px.bar(dates_count_df, x=dates_count_df.index, y="Participants")
fig.show()
fig.write_image("images/Time Series.png")




print(dates_count_df.sort_index())


            Participants
Date                    
2022-10-04            12
2022-10-05             2
2022-10-06             3
2022-10-07            62
2022-10-08            11
2022-10-09            14
2022-10-10            11
2022-10-11            10
2022-10-12             4
2022-10-13            17
2022-10-14            14
2022-10-15            15
2022-10-16             8
2022-10-17             1
2022-10-18             2
2022-10-19             3
2022-10-24             2
2022-10-26            15
2022-10-27             9
2022-10-31             1
2022-11-08             1
2022-11-09             1
2022-11-10             1
2022-11-14             3
2022-11-15            14
2022-11-16             2
2022-11-17             1
